In [17]:
import numpy as np
import tensorflow as tf
from zipfile import ZipFile
import tensorflow_io as tfio
from matplotlib import pyplot as plt
import os

In [18]:
def load_wav_16k_mono(filename):
  file_contents = tf.io.read_file(filename)
  wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
  wav = tf.squeeze(wav, axis=-1)
  sample_rate = tf.cast(sample_rate, dtype=tf.int64)
  wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000) # TODO: Verificar se nessa qualidade está ok
  return wav

In [19]:
PIANO = os.path.join('data', 'pia') # , '*.wav')
GUITAR = os.path.join('data', 'gac')
VIOLIN = os.path.join('data', 'vio')

In [21]:
# Tensor com caminho dos arquivos
piano = tf.data.Dataset.list_files(PIANO + '/*.wav')
guitar = tf.data.Dataset.list_files(GUITAR + '/*.wav')
violin = tf.data.Dataset.list_files(VIOLIN + '/*.wav')

In [22]:
pianoLab = tf.data.Dataset.zip(piano, tf.data.Dataset.from_tensor_slices(
    tf.keras.utils.to_categorical(tf.fill(len(piano), 0), num_classes=3) # Label [1,0,0] para piano
))
guitarLab = tf.data.Dataset.zip(guitar, tf.data.Dataset.from_tensor_slices(
    tf.keras.utils.to_categorical(tf.fill(len(guitar), 1), num_classes=3) # Label [0,1,0] para violão
))
violinLab = tf.data.Dataset.zip(violin, tf.data.Dataset.from_tensor_slices(
    tf.keras.utils.to_categorical(tf.fill(len(violin), 2), num_classes=3) # Label [0,0,1] para violino
))

data = pianoLab.concatenate(guitarLab)
data = data.concatenate(violinLab)

In [23]:
def preprocess(file_path, label):
  wav = load_wav_16k_mono(file_path)

  spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32) # TODO: Verificar frame_length
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=2)
  return spectrogram, label

In [24]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=10000)
data = data.batch(16)
data = data.prefetch(8)

In [9]:
train = data.take(86) # 86 +- 70% de 122
test = data.skip(86).take(36) # 36 restantes para teste

# Treinando modelo

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

In [11]:
# Projetando modelo:

rnnModel = Sequential()
rnnModel.add(LSTM(64, input_shape=(1490, 257), return_sequences=True))
rnnModel.add(LSTM(64))
#model.add(layers.Flatten())
rnnModel.add(Dense(64, activation='relu'))
#rnnModel.add(Dropout(0.3))

rnnModel.add(Dense(3, activation='softmax')) # 3 é o número de classes da resposta

In [12]:
rnnModel.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
rnnModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1490, 64)          82432     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 119811 (468.01 KB)
Trainable params: 119811 (468.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
hist2 = rnnModel.fit(train, epochs=4, validation_data=test, batch_size=16)

Epoch 1/4


TypeError: in user code:

    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/racklyn/.local/lib/python3.10/site-packages/keras/src/layers/rnn/lstm.py", line 616, in call
        ti